# EMA Cross Strategy Example

This notebook demonstrates how to use the C++ Backtest Engine with a simple EMA Crossover strategy.

In [ ]:
import sys
sys.path.append('../python')
from backtester import Backtester, Strategy, Bar, Side
import random
import time

In [ ]:
class EmaCrossStrategy(Strategy):
    def __init__(self, engine, fast_period=10, slow_period=20):
        super().__init__(engine)
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.prices = []
        
    def on_bar(self, bar: Bar):
        self.prices.append(bar.close)
        
        if len(self.prices) < self.slow_period:
            return
            
        # Calculate EMAs (using GPU for demonstration if list is long, but here simple CPU)
        # For demo, we just use simple average of last N
        fast_ma = sum(self.prices[-self.fast_period:]) / self.fast_period
        slow_ma = sum(self.prices[-self.slow_period:]) / self.slow_period
        
        # Trading Logic
        pos_qty = self.engine.get_portfolio().get_position_quantity(bar.instrument_id)
        
        if fast_ma > slow_ma and pos_qty <= 0:
            print(f"Buy at {bar.close}")
            self.buy(bar.instrument_id, 100)
        elif fast_ma < slow_ma and pos_qty >= 0:
            print(f"Sell at {bar.close}")
            self.sell(bar.instrument_id, 100)

In [ ]:
# Run Simulation
backtester = Backtester()
strategy = backtester.add_strategy(EmaCrossStrategy)

# Generate dummy data
bars = []
price = 100.0
for i in range(1000):
    price += (random.random() - 0.5)
    bar = Bar()
    bar.timestamp = i * 60 * 1000000000 # nanoseconds
    bar.instrument_id = 1
    bar.close = price
    bar.open = price
    bar.high = price
    bar.low = price
    bar.volume = 1000
    bars.append(bar)

backtester.add_data(bars)

start = time.time()
backtester.run()
end = time.time()

print(f"Simulation took {end - start:.4f}s")
print(f"Final Equity: {backtester.get_portfolio().get_equity()}")